**What Is Fake News ?**

False news, also known as unwanted news, false stories, misconceptions or fraudulent stories, types of stories that contain deliberate information or frauds that are spread through traditional media (print and broadcast) or online media. Digital news has revived and increased the use of fake news, or yellow journalism. These stories are often referred to as information that is not in the media but sometimes finds its way into the mainstream media as well.


*   It causes panic
*   Damaging the reputation of public and private organizations
*   It deceives the people, for the benefit of the deceivers
*   Motivated by a personal vendetta, some people support such things.

<br>
<br>

**PROBLEM :** 
<br>
How to distinguish between a real news and a fake news?

<br>
<br>

**SOLUTION :**

We can show an algorithm that has a large number of false and real news stories to learn to distinguish between them automatically, and then give a possible score or percentage of confidence such as a given news release is true or false.

# Exploratory Data Analysis

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import pandas as pd
import numpy as np

**Loading The Dataset**

In [3]:
f = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv', delimiter = ',')
t = pd.read_csv('../input/fake-and-real-news-dataset/True.csv', delimiter = ',')

**Observing the top 5 rows of fake and true data**

In [4]:
f.head()

In [5]:
t.head()

Now combining these two dataset to one dataset to simplify processing.

Also to combine we need to add an extra column as 'temp' to differtiate news as 1=true_news 0=fake_news

In [6]:
f['temp']= 0
t['temp']= 1

datas = pd.DataFrame()
datas = t.append(f)

**Observing the top 5 rows of dataset**

In [7]:
datas.head()

**Checking the dimensions**

In [8]:
print(datas.shape)

**Column 'Date' and 'Subject' are important to Descriptive analysis but here for prediction they are less important so dropping these columns.**

In [9]:
column = ['date','subject']
datas = datas.drop(columns=column)

**Cheking the dimensions after dropping**

In [10]:
print(datas.shape)

**Exploring the data**

In [11]:
datas.info()

**Created array of 'title' column as input_array for preprocessing**

In [12]:
input_arr=np.array(datas['title'])

# Natural Language Processing

*Downloading Stopwords*

In [ ]:
import re
import nltk
nltk.download('stopwords')

***Stopwords***: *A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore, both when indexing entries for searching and when retrieving them as the result of a search query.*
https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

***PortStemmer***: *A PortStemmer is an algorithm used for removing the commoner morphological and inflexional endings from words in English.  For example: words such as “Likes”, ”liked”, ”likely” and ”liking” will be reduced to “like” after stemming.*
https://www.geeksforgeeks.org/python-stemming-words-with-nltk/

**Using nltk and importing Stopwords(For dealing with stopwords) and PortStemmer(For stemming)**

In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []

for i in range(0, 40000):
    newArr = re.sub('[^a-zA-Z]', ' ', input_arr[i])
    #The sub() function replaces the matches with the text of your choice,
    #in this case "[a-zA-Z]" is getting replaced with blank space
    #input_arr[i] is the array of 'title' column.

    #[a-zA-Z]: Returns a match for any character alphabetically between a and z, lower case OR upper case

    newArr = newArr.lower()
    #Converting into lowercase

    newArr = newArr.split()
    #The split() method splits a string into a list.

    ps = PorterStemmer()
    newArr = [ps.stem(word) for word in newArr if not word in set(stopwords.words('english'))]
    #ps.stem(word) is stemming the words
    #the word will be considered if the word is not a stopword
    #set(stopwords.words('english')= checks any kind of stopwords in English language

    newArr = ' '.join(newArr)
    #joins the string with blank spaces

    corpus.append(newArr)
    #adding strings into the corpus list

In [15]:
datas

***CountVectorizer*** : *CountVectorizer is a tool provided by the scikit-learn library. It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.* 
https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
countv = CountVectorizer(max_features = 5000)
#max_features: The CountVectorizer will select the
#words/features/terms which occur the most frequently.
#It takes absolute values so if you set the ‘max_features = 3’,
#it will select the 3 most common words in the data.

X = countv.fit_transform(corpus).toarray()
y = datas.iloc[0:40000, 2].values

X=independent_variable(title)
</br> y=dependent_variable(sentiment 0 or 1)

</br>
Basically what I did here is created a bag of words (https://machinelearningmastery.com/gentle-introduction-bag-words-model/)i.e, X will have top 5000 most common words in array form

# Modelling

**Splitting the dataset into the Training set and Test set in 80-20 ratio**

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

**Fitting Logistic Regression to the Training set**

In [18]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

**Predicting the Test set results**

In [19]:
y_pred = classifier.predict(X_test)

# Model Evaluation

**Making the Confusion Matrix**

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [21]:
print(cm)

**Getting more details**

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

TP – True Positives
TN - True Negatives
FP – False Positives
FN – False Negatives

Precision: Accuracy of positive predictions.
Precision = TP/(TP + FP)

Recall: Fraction of positives that were correctly identified.
Recall = TP/(TP+FN)

F1 score – Percent of positive predictions were correct.
F1 Score = 2*(Recall * Precision) / (Recall + Precision)

**Getting Accuracy**

In [23]:
from sklearn.metrics import accuracy_score
print("Accuarcy: {}".format(round(accuracy_score(y_test, y_pred)*100,2)))

**Confusion matrix in a well plotted chart**

In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
new_cm = pd.DataFrame(cm , index = ['Fake','Not Fake'] , columns = ['Fake','Not Fake'])
sns.heatmap(new_cm,cmap= 'Blues', annot = True, fmt='',xticklabels = ['Fake','Not Fake'], yticklabels = ['Fake','Not Fake'])
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title('Confusion matrix On Test Data')
plt.show()